In [1]:
import pandas as pd

In [16]:
import numpy as np
from datetime import timedelta, datetime, time

In [5]:
task = pd.read_excel(r'C:\2025AUTUMN\MT\25HS_MT_Joy\25HS_MT_Joy\Data\Timbeco\Rapla-design\411_Spetsifikatsioonid\NP2424_PP_EK-9-101_v02_Rapla-elemendid-koond.xlsx', skiprows = 5, header = 0)

In [6]:
print(task.head())

   Unnamed: 0    Pos NR  Profiil / Profile  Pikkus [mm] / Length [mm]  \
0         NaN  KL-02-29                NaN                       6960   
1         NaN  KL-02-30                NaN                       6960   
2         NaN  KL-02-31                NaN                       6960   
3         NaN  KL-02-32                NaN                       6960   
4         NaN  KL-02-33                NaN                       6960   

   Laius [mm] / Width [mm]  Kõrgus / Height [mm]  Arv [tk] / Pcs  \
0                      554                  3135               1   
1                      554                  2434               1   
2                      554                  2520               1   
3                      554                  2785               1   
4                      554                  3258               1   

   1tk Kaal [kg] / 1 piece weight [kg]  Kaal kokku [kg] / Total weight [kg]  \
0                               1016.3                               1016

In [7]:
task_id = task['Pos NR'].tolist()

In [8]:
task_type = ['Procurement', 'CNC', 'Element Assembly', 'Window Installation', 'Transport', 'Installation Work','Crane Work', 'Facade Work', 'Interior Work', 'Roof Work']

In [12]:
df = pd.DataFrame(columns=['Task ID', 'Task Type', 'Plan Start Date', 'Plan End Date', 'Plan Duration', 'Actual Start Date', 'Actual End Date', 'Actual Duration', 'Complete %', 'Slack Day', 'Risk', 'Predecessors', 'Difference'])

In [13]:
df['Task ID'] = task_id

In [14]:
# task type for all rows are set to be 'Installation Work'
df['Task Type'] = 'Installation Work'
df.head()

,Task ID,Task Type,Plan Start Date,Plan End Date,Plan Duration,Actual Start Date,Actual End Date,Actual Duration,Complete %,Slack Day,Risk,Predecessors,Difference
0,KL-02-29,Installation Work,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KL-02-30,Installation Work,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KL-02-31,Installation Work,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,KL-02-32,Installation Work,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,KL-02-33,Installation Work,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def next_business_start(dt, work_start: time, work_end: time):

    dt = pd.Timestamp(dt)
    while dt.weekday() >= 5:  # 5=Sat,6=Sun
        dt = pd.Timestamp(dt.date() + timedelta(days=1))
    if dt.time() < work_start:
        return pd.Timestamp.combine(dt.date(), work_start)
    if dt.time() >= work_end:
        # jump to next business day
        d = dt.date() + timedelta(days=1)
        while d.weekday() >= 5:
            d += timedelta(days=1)
        return pd.Timestamp.combine(d, work_start)
    return dt

In [18]:
def add_business_hours(start_dt, hours: float, 
                       work_start=time(9,0), work_end=time(17,0)):

    cur = next_business_start(start_dt, work_start, work_end)
    remain = float(hours)
    work_span = (datetime.combine(cur.date(), work_end) -
                 datetime.combine(cur.date(), work_start)).total_seconds() / 3600.0
    if work_span <= 0:
        raise ValueError("work_end must be after work_start")

    while remain > 1e-9:
        cur = next_business_start(cur, work_start, work_end)
        end_today = pd.Timestamp.combine(cur.date(), work_end)
        available = (end_today - cur).total_seconds()/3600.0
        if available <= 0:
            cur = cur + timedelta(days=1)  # 到下一天再由 next_business_start 纠正
            continue
        use = min(remain, available)
        cur += timedelta(hours=use)
        remain -= use
        if remain > 1e-9:
            # 跳到下一工作日开工
            cur = pd.Timestamp.combine(cur.date() + timedelta(days=1), work_start)
    return cur


In [35]:
def fill_plan_start_and_duration( 
    df: pd.DataFrame,
    project_start="2025-10-20 09:00",
    mode="serial",                 # "serial" | "parallel"
    work_start="09:00",
    work_end="17:00",
    min_hours=0,
    max_hours=10,
    random_seed=None
):
    out = df.copy()

    # 用随机整数(小时)直接覆盖 Plan Duration
    rng = np.random.default_rng(random_seed)
    out["Plan Duration"] = rng.integers(min_hours, max_hours + 1, size=len(out)).astype(int)

    start0 = pd.to_datetime(project_start)
    ws_h, ws_m = map(int, work_start.split(":"))
    we_h, we_m = map(int, work_end.split(":"))
    ws, we = time(ws_h, ws_m), time(we_h, we_m)

    if mode == "parallel":
        start = next_business_start(start0, ws, we)  
        out["Plan Start Date"] = start
    else:  # serial
        starts = []
        durations = []
        ends = []
        cur = start0
        for h in out["Plan Duration"]:
            cur = next_business_start(cur, ws, we)          
            starts.append(cur)
            cur = add_business_hours(cur, float(h), ws, we)
            durations.append(float(h))
            ends.append(cur)
        out["Plan Start Date"] = starts
        out["Plan Duration"] = durations
        out["Plan End Date"] = ends

    return out

In [36]:
out = fill_plan_start_and_duration(df, project_start="2025-10-20 09:00", mode="serial")

In [37]:
print(out.head())

    Task ID          Task Type     Plan Start Date       Plan End Date  \
0  KL-02-29  Installation Work 2025-10-20 09:00:00 2025-10-21 10:00:00   
1  KL-02-30  Installation Work 2025-10-21 10:00:00 2025-10-22 10:00:00   
2  KL-02-31  Installation Work 2025-10-22 10:00:00 2025-10-22 11:00:00   
3  KL-02-32  Installation Work 2025-10-22 11:00:00 2025-10-23 13:00:00   
4  KL-02-33  Installation Work 2025-10-23 13:00:00 2025-10-24 15:00:00   

   Plan Duration Actual Start Date Actual End Date Actual Duration Complete %  \
0            9.0               NaN             NaN             NaN        NaN   
1            8.0               NaN             NaN             NaN        NaN   
2            1.0               NaN             NaN             NaN        NaN   
3           10.0               NaN             NaN             NaN        NaN   
4           10.0               NaN             NaN             NaN        NaN   

  Slack Day Risk Predecessors Difference  
0       NaN  NaN         

In [38]:
out['Risk'] = 'Low'

In [45]:
out.to_csv(r'C:\2025AUTUMN\MT\25HS_MT_Joy\25HS_MT_Joy\Data\Timbeco\input_file.csv', index=False)

In [40]:
from sqlalchemy import create_engine

In [41]:
engine = create_engine('sqlite:///input_database.db')

In [42]:
out.to_sql('input_table', con=engine, if_exists='replace', index=False)

151

In [43]:
print(out.head())

    Task ID          Task Type     Plan Start Date       Plan End Date  \
0  KL-02-29  Installation Work 2025-10-20 09:00:00 2025-10-21 10:00:00   
1  KL-02-30  Installation Work 2025-10-21 10:00:00 2025-10-22 10:00:00   
2  KL-02-31  Installation Work 2025-10-22 10:00:00 2025-10-22 11:00:00   
3  KL-02-32  Installation Work 2025-10-22 11:00:00 2025-10-23 13:00:00   
4  KL-02-33  Installation Work 2025-10-23 13:00:00 2025-10-24 15:00:00   

   Plan Duration Actual Start Date Actual End Date Actual Duration Complete %  \
0            9.0               NaN             NaN             NaN        NaN   
1            8.0               NaN             NaN             NaN        NaN   
2            1.0               NaN             NaN             NaN        NaN   
3           10.0               NaN             NaN             NaN        NaN   
4           10.0               NaN             NaN             NaN        NaN   

  Slack Day Risk Predecessors Difference  
0       NaN  Low         